In [1]:
!pip install openai

In [ ]:
import joblib
import openai

# Load the churn predictions model
churn_model = joblib.load("./../models/model.joblib")

# Set up the OpenAI API credentials
openai.api_key = "YOUR_KEY"

# Function to generate personalized retention incentives
def generate_retention_incentives(customer_data):
    # Make churn prediction using the churn model
    churn_probability = churn_model.predict_proba(customer_data)[0][1]

    # Generate personalized retention incentive using OpenAI GPT-3.5 Turbo
    prompt = f"Customer churn probability: {churn_probability}\nCustomer data: {customer_data}\nGenerate personalized retention incentive:"

    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.5,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
    )

    retention_incentive = response.choices[0].text.strip()

    return retention_incentive

# Example usage
customer_data = {
    "age": 30,
    "gender": "male",
    "purchase_history": ["product1", "product2", "product3"],
    # Include other relevant customer data
}

incentive = generate_retention_incentives(customer_data)
print("Retention Incentive:")
print(incentive)